# 基礎設定

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, glob, json, math, shutil, zipfile, random, time, csv
import numpy as np
import matplotlib.pyplot as plt
import cv2
from collections import Counter, defaultdict
from dataclasses import dataclass
from tqdm import tqdm
from pathlib import Path
import yaml

In [ ]:
import torch
torch.backends.cudnn.benchmark = True  # 加速

In [ ]:
# 關 W&B
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
SEED = 42
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
seed_everything(SEED)

In [ ]:
ZIP_PATH = "/content/drive/MyDrive/ComputerVision_DL/hw2_longtail_od/taica-cvpdl-2025-hw-2.zip"
EXTRACT_DIR = "/content/data_hw2"
REPORT_DIR  = "/content/reports"
os.makedirs(EXTRACT_DIR, exist_ok=True)
os.makedirs(REPORT_DIR, exist_ok=True)

# 提交輸出
SUBMIT_DIR = "/content/drive/MyDrive/ComputerVision_DL/hw2_longtail_od/submissions"
os.makedirs(SUBMIT_DIR, exist_ok=True)

# 解壓 & 掃描資料

In [ ]:
def find_dataset_root(root):
    for dirpath, dirnames, filenames in os.walk(root):
        if 'train' in dirnames and 'test' in dirnames:
            return dirpath
    candidate_trains = []
    for dirpath, dirnames, filenames in os.walk(root):
        if os.path.basename(dirpath).lower() == 'train':
            has_txt = any(fn.lower().endswith('.txt') for fn in filenames)
            has_png = any(fn.lower().endswith('.png') for fn in filenames)
            if has_txt and has_png:
                candidate_trains.append(dirpath)
    candidate_trains = sorted(candidate_trains, key=lambda d: len(d.split(os.sep)), reverse=True)
    for tdir in candidate_trains:
        parent = os.path.dirname(tdir)
        tdir_sibling_test = os.path.join(parent, 'test')
        if os.path.isdir(tdir_sibling_test):
            return parent
    return None

In [ ]:
with zipfile.ZipFile(ZIP_PATH, 'r') as zf:
    zf.extractall(EXTRACT_DIR)

In [ ]:
DATASET_ROOT = find_dataset_root(EXTRACT_DIR)
if DATASET_ROOT is None:
    raise RuntimeError(f"找不到包含 train/test 的資料夾，請檢查 {EXTRACT_DIR} 的解壓內容結構")

TRAIN_DIR = os.path.join(DATASET_ROOT, 'train')
TEST_DIR  = os.path.join(DATASET_ROOT, 'test')

In [ ]:
def sorted_by_stem(paths):
    return sorted(paths, key=lambda p: os.path.splitext(os.path.basename(p))[0])

TRAIN_IMGS = sorted_by_stem(glob.glob(os.path.join(TRAIN_DIR, "*.png")))
TRAIN_TXTS = sorted_by_stem(glob.glob(os.path.join(TRAIN_DIR, "*.txt")))
TEST_IMGS  = sorted_by_stem(glob.glob(os.path.join(TEST_DIR,  "*.png")))


In [ ]:
def stem(p): return os.path.splitext(os.path.basename(p))[0]
train_img_by_stem = {stem(p): p for p in TRAIN_IMGS}
train_lab_by_stem = {stem(p): p for p in TRAIN_TXTS}

In [ ]:
COMMON_KEYS = sorted(set(train_img_by_stem) & set(train_lab_by_stem))
ONLY_IMG    = sorted(set(train_img_by_stem) - set(train_lab_by_stem))
ONLY_TXT    = sorted(set(train_lab_by_stem) - set(train_img_by_stem))

print(f"DATASET_ROOT: {DATASET_ROOT}")
print(f"TRAIN_DIR   : {TRAIN_DIR}")
print(f"TEST_DIR    : {TEST_DIR}")
print(f"Train images: {len(TRAIN_IMGS)} | Train labels: {len(TRAIN_TXTS)} | Test images: {len(TEST_IMGS)}")
print(f"Paired train files: {len(COMMON_KEYS)} | only_img: {len(ONLY_IMG)} | only_txt: {len(ONLY_TXT)}")


DATASET_ROOT: /content/data_hw2/CVPDL_hw2/CVPDL_hw2
TRAIN_DIR   : /content/data_hw2/CVPDL_hw2/CVPDL_hw2/train
TEST_DIR    : /content/data_hw2/CVPDL_hw2/CVPDL_hw2/test
Train images: 950 | Train labels: 950 | Test images: 550
Paired train files: 950 | only_img: 0 | only_txt: 0


In [ ]:
# =========================
# 2) 基礎工具 & 清理規則
# =========================

In [ ]:
CLASSES = ["car", "hov", "person", "motorcycle"]

In [ ]:
@dataclass
class BBox:
    cls: int; x: int; y: int; w: int; h: int

def imread_rgb(p):
    im = cv2.imread(p, cv2.IMREAD_COLOR)
    if im is None: return None
    return cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

def parse_label_line(line):
    parts = [x.strip() for x in line.split(",")]
    if len(parts) != 5:
        return None
    try:
        c, x, y, w, h = map(int, parts)
        return BBox(c, x, y, w, h)
    except Exception:
        return None

def load_label_txt(txt_path):
    items, bad_lines = [], []
    with open(txt_path, "r") as f:
        for ln, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            bb = parse_label_line(line)
            if bb is None:
                bad_lines.append((ln, line))
            else:
                items.append(bb)
    return items, bad_lines

DIM_CACHE = {}
def get_hw(img_path):
    if img_path in DIM_CACHE:
        return DIM_CACHE[img_path]
    im = imread_rgb(img_path)
    if im is None:
        return None
    h, w = im.shape[:2]
    DIM_CACHE[img_path] = (h, w)
    return h, w

def iou_xywh(a, b):
    ax1, ay1, ax2, ay2 = a.x, a.y, a.x + a.w, a.y + a.h
    bx1, by1, bx2, by2 = b.x, b.y, b.x + b.w, b.y + b.h
    ix1, iy1 = max(ax1, bx1), max(ay1, by1)
    ix2, iy2 = min(ax2, bx2), min(ay2, by2)
    iw, ih = max(0, ix2 - ix1), max(0, iy2 - iy1)
    inter = iw * ih
    if inter <= 0: return 0.0
    a_area = a.w * a.h
    b_area = b.w * b.h
    union = a_area + b_area - inter + 1e-6
    return inter / union

def clip_bbox(bb, W, H):
    x1 = max(0, min(bb.x, W-1))
    y1 = max(0, min(bb.y, H-1))
    x2 = max(0, min(bb.x + bb.w, W))
    y2 = max(0, min(bb.y + bb.h, H))
    w = max(0, x2 - x1)
    h = max(0, y2 - y1)
    return BBox(bb.cls, x1, y1, w, h)


In [ ]:
MIN_SIDE_PX = 2
MIN_AREA_PX = 4
VALID_CLASS_IDS = set(range(len(CLASSES)))
DUP_IOU_THR = 0.9

In [ ]:
def dedup_by_iou(items, thr=0.9):
    kept = []
    items_sorted = sorted(items, key=lambda b: b.w*b.h, reverse=True)
    for bb in items_sorted:
        drop = False
        for kk in kept:
            if bb.cls == kk.cls and iou_xywh(bb, kk) > thr:
                drop = True; break
        if not drop:
            kept.append(bb)
    return kept


In [ ]:
# =========================
# 3) 報表與清理到 labels_clean（路徑統一）
# =========================

In [ ]:
bad_rows = []; invalid_cls = []; nonpos_wh = []; oob_boxes = []; tiny_boxes = []; dup_boxes = []
per_image_class_count = defaultdict(lambda: Counter())
size_buckets = Counter()

In [ ]:
def bucket_by_area_ratio(w, h, W, H):
    ratio = (w*h) / float(W*H + 1e-6)
    if ratio < 0.001: return "small"
    elif ratio < 0.01: return "medium"
    else: return "large"

for k in COMMON_KEYS:
    img_p, txt_p = train_img_by_stem[k], train_lab_by_stem[k]
    dim = get_hw(img_p)
    if dim is None:
        bad_rows.append([k, "image_read_fail", "", ""]); continue
    H,W = dim
    items, bads = load_label_txt(txt_p)
    for (ln, line) in bads:
        bad_rows.append([k, "parse_fail", ln, line])
    for it in items:
        per_image_class_count[k][it.cls] += 1
        if it.cls not in VALID_CLASS_IDS:
            invalid_cls.append([k, it.cls])
        if it.w <= 0 or it.h <= 0:
            nonpos_wh.append([k, it.cls, it.x, it.y, it.w, it.h])
        if not (0 <= it.x < W and 0 <= it.y < H and it.x + it.w <= W + 1 and it.y + it.h <= H + 1):
            oob_boxes.append([k, it.cls, it.x, it.y, it.w, it.h, W, H])
        if it.w < MIN_SIDE_PX or it.h < MIN_SIDE_PX or (it.w * it.h) < MIN_AREA_PX:
            tiny_boxes.append([k, it.cls, it.x, it.y, it.w, it.h])
        else:
            size_buckets[bucket_by_area_ratio(it.w, it.h, W, H)] += 1
    for c in set([it.cls for it in items]):
        cls_items = [it for it in items if it.cls == c and it.w > 0 and it.h > 0]
        n = len(cls_items)
        for i in range(n):
            for j in range(i+1, n):
                if iou_xywh(cls_items[i], cls_items[j]) > DUP_IOU_THR:
                    dup_boxes.append([k, c, i, j])

def write_csv(path, header, rows):
    with open(path, "w", newline="") as f:
        w = csv.writer(f); w.writerow(header); w.writerows(rows)


In [ ]:
write_csv(os.path.join(REPORT_DIR, "bad_rows.csv"), ["stem","reason","line_no","raw_line"], bad_rows)
write_csv(os.path.join(REPORT_DIR, "invalid_class.csv"), ["stem","cls"], invalid_cls)
write_csv(os.path.join(REPORT_DIR, "nonpos_wh.csv"), ["stem","cls","x","y","w","h"], nonpos_wh)
write_csv(os.path.join(REPORT_DIR, "out_of_bounds.csv"), ["stem","cls","x","y","w","h","W","H"], oob_boxes)
write_csv(os.path.join(REPORT_DIR, "tiny_boxes.csv"), ["stem","cls","x","y","w","h"], tiny_boxes)
write_csv(os.path.join(REPORT_DIR, "duplicate_boxes.csv"), ["stem","cls","idx_i","idx_j"], dup_boxes)


In [ ]:
class_counts = Counter()
for k, cnt in per_image_class_count.items():
    for c, v in cnt.items():
        class_counts[c] += v

stats = {
    "class_counts": {CLASSES[c]: int(class_counts[c]) for c in range(len(CLASSES))},
    "size_buckets": dict(size_buckets),
    "num_train_pairs": len(COMMON_KEYS),
    "only_img": len(ONLY_IMG),
    "only_txt": len(ONLY_TXT),
}
with open(os.path.join(REPORT_DIR, "dataset_stats.json"), "w") as f:
    json.dump(stats, f, indent=2)
print(json.dumps(stats, indent=2))
print(f"Reports saved under: {REPORT_DIR}")

{
  "class_counts": {
    "car": 23275,
    "hov": 1345,
    "person": 3363,
    "motorcycle": 5348
  },
  "size_buckets": {
    "small": 23201,
    "medium": 9891,
    "large": 239
  },
  "num_train_pairs": 950,
  "only_img": 0,
  "only_txt": 0
}
Reports saved under: /content/reports


In [ ]:
# 清理到 DATASET_ROOT/labels_clean（路徑統一）
CLEAN_LAB_DIR = os.path.join(DATASET_ROOT, "labels_clean")
os.makedirs(CLEAN_LAB_DIR, exist_ok=True)

In [ ]:
num_changed = 0
for k in COMMON_KEYS:
    img_p, txt_p = train_img_by_stem[k], train_lab_by_stem[k]
    dim = get_hw(img_p)
    if dim is None: continue
    H, W = dim
    items, _ = load_label_txt(txt_p)
    fixed = []
    for it in items:
        it2 = clip_bbox(it, W, H)
        if it2.w <= 0 or it2.h <= 0: continue
        if it2.w < MIN_SIDE_PX or it2.h < MIN_SIDE_PX: continue
        fixed.append(it2)
    fixed2 = dedup_by_iou(fixed, thr=DUP_IOU_THR)
    out_p = os.path.join(CLEAN_LAB_DIR, f"{k}.txt")
    with open(out_p, "w") as f:
        for b in fixed2:
            f.write(f"{b.cls},{b.x},{b.y},{b.w},{b.h}\n")
    if len(fixed2) != len(items): num_changed += 1

print(f"Cleaned labels saved under: {CLEAN_LAB_DIR} | files_changed: {num_changed}")


Cleaned labels saved under: /content/data_hw2/CVPDL_hw2/CVPDL_hw2/labels_clean | files_changed: 0


## 轉 YOLO 標註格式（cx cy w h, 0~1）

In [ ]:
USE_LAB_DIR = CLEAN_LAB_DIR if len(glob.glob(os.path.join(CLEAN_LAB_DIR, "*.txt"))) > 0 else TRAIN_DIR
print("Using labels from:", USE_LAB_DIR, "files=", len(glob.glob(os.path.join(USE_LAB_DIR, "*.txt"))))


Using labels from: /content/data_hw2/CVPDL_hw2/CVPDL_hw2/labels_clean files= 950


In [ ]:
YOLO_ROOT = "/content/yolo_dataset"
IMG_DIR   = os.path.join(YOLO_ROOT, "images")
LAB_DIR   = os.path.join(YOLO_ROOT, "labels")
shutil.rmtree(YOLO_ROOT, ignore_errors=True)
os.makedirs(IMG_DIR, exist_ok=True); os.makedirs(LAB_DIR, exist_ok=True)


In [ ]:
_DIM_CACHE = {}
def get_hw_fast(img_path):
    if img_path in _DIM_CACHE: return _DIM_CACHE[img_path]
    im = cv2.imread(img_path, cv2.IMREAD_COLOR)
    if im is None: return None
    h, w = im.shape[:2]
    _DIM_CACHE[img_path] = (h, w)
    return h, w

def load_items_txt(txt_path):
    items = []
    with open(txt_path, "r") as f:
        for line in f:
            line=line.strip()
            if not line: continue
            parts=[x.strip() for x in line.split(",")]
            if len(parts)!=5: continue
            c,x,y,w,h = map(int, parts)
            items.append((c,x,y,w,h))
    return items

num_ok, num_skip = 0, 0
for k in tqdm(COMMON_KEYS, desc="Convert to YOLO labels"):
    src_img = train_img_by_stem[k]
    src_lab = os.path.join(USE_LAB_DIR, f"{k}.txt")
    if not os.path.isfile(src_img) or not os.path.isfile(src_lab):
        num_skip += 1; continue
    hw = get_hw_fast(src_img)
    if hw is None: num_skip += 1; continue
    H, W = hw
    dst_img = os.path.join(IMG_DIR, f"{k}.png")
    if not os.path.isfile(dst_img):
        shutil.copy2(src_img, dst_img)
    items = load_items_txt(src_lab)
    yolo_lines = []
    for (c,x,y,w,h) in items:
        if w <= 0 or h <= 0: continue
        cx, cy = x + w/2, y + h/2
        yolo_cx = min(1, max(0, cx / W))
        yolo_cy = min(1, max(0, cy / H))
        yolo_w  = min(1, max(0, w / W))
        yolo_h  = min(1, max(0, h / H))
        yolo_lines.append(f"{c} {yolo_cx:.6f} {yolo_cy:.6f} {yolo_w:.6f} {yolo_h:.6f}")
    with open(os.path.join(LAB_DIR, f"{k}.txt"), "w") as f:
        f.write("\n".join(yolo_lines))
    num_ok += 1

print(f"Converted: {num_ok} | Skipped: {num_skip}")
print("YOLO dataset root:", YOLO_ROOT)


Convert to YOLO labels: 100%|██████████| 950/950 [00:51<00:00, 18.41it/s]

Converted: 950 | Skipped: 0
YOLO dataset root: /content/yolo_dataset


In [ ]:
# =========================
# 5) 5 折（多標籤分層）＋ Head-only 下採樣 ＋ RFS
# =========================

In [ ]:
!pip -q install scikit-multilearn==0.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 8.8 MB/s eta 0:00:00


In [ ]:
from skmultilearn.model_selection import IterativeStratification

In [ ]:
presence_vector = {}
cls_counts_per_img = {}
for k in COMMON_KEYS:
    lab_p = os.path.join(LAB_DIR, f"{k}.txt")
    cnt = Counter(); vec = [0]*len(CLASSES)
    if os.path.isfile(lab_p):
        with open(lab_p, "r") as f:
            for line in f:
                line=line.strip()
                if not line: continue
                cid=int(line.split()[0])
                cnt[cid]+=1; vec[cid]=1
    cls_counts_per_img[k]=cnt; presence_vector[k]=vec

X = list(COMMON_KEYS)
Y = np.array([presence_vector[k] for k in X], dtype=int)
mskf = IterativeStratification(n_splits=5, order=1)
folds=[]
for tr_idx, va_idx in mskf.split(X, Y):
    tr_keys = [X[i] for i in tr_idx]
    va_keys = [X[i] for i in va_idx]
    folds.append({"train": tr_keys, "val": va_keys})
with open(os.path.join(REPORT_DIR, "folds_5.json"), "w") as f:
    json.dump(folds, f, indent=2)
print("Saved 5 multilabel folds ->", os.path.join(REPORT_DIR, "folds_5.json"))


Saved 5 multilabel folds -> /content/reports/folds_5.json


In [ ]:
# Head-only 下採樣（僅含 car 的影像）
HEAD_ONLY = [k for k,v in presence_vector.items() if v[0]==1 and sum(v[1:])==0]
HEAD_KEEP = 0.5

def downsample_head_only(keys):
    heads = [k for k in keys if k in HEAD_ONLY]
    others= [k for k in keys if k not in HEAD_ONLY]
    keep_n = int(len(heads) * HEAD_KEEP)
    kept_head = random.sample(heads, keep_n) if keep_n < len(heads) else heads
    return others + kept_head

for f in folds:
    f["train"] = downsample_head_only(f["train"])


In [ ]:
# RFS
global_cls_counts = Counter()
for k in COMMON_KEYS:
    global_cls_counts.update(cls_counts_per_img[k])
eps=1e-9
total_boxes = sum(global_cls_counts.values()) + eps
freq = {c:(global_cls_counts[c]/total_boxes) for c in range(len(CLASSES))}
repeat_thresh_per_class = {1:0.006, 2:0.010, 3:0.008}  # hov/person/motorcycle
BASE_T = 0.002
RFS_CAP = 3.0

def image_repeat_factor(k):
    max_r = 1.0
    for c, n in cls_counts_per_img[k].items():
        t = repeat_thresh_per_class.get(c, BASE_T)
        fc = freq.get(c, 1e-9)
        r = math.sqrt(t / max(fc, 1e-9))
        max_r = max(max_r, r)
    return min(max_r, RFS_CAP)

LIST_DIR = os.path.join(YOLO_ROOT, "splits")
os.makedirs(LIST_DIR, exist_ok=True)

def img_path_of(k): return os.path.join(IMG_DIR, f"{k}.png")

for fi, fold in enumerate(folds):
    tr, va = fold["train"], fold["val"]
    with open(os.path.join(LIST_DIR, f"fold{fi}_train.txt"), "w") as f:
        for k in tr: f.write(img_path_of(k) + "\n")
    with open(os.path.join(LIST_DIR, f"fold{fi}_val.txt"), "w") as f:
        for k in va: f.write(img_path_of(k) + "\n")
    with open(os.path.join(LIST_DIR, f"fold{fi}_train_rfs.txt"), "w") as f:
        for k in tr:
            n_rep = int(round(image_repeat_factor(k)))
            for _ in range(max(1, n_rep)): f.write(img_path_of(k) + "\n")

print("Generated split lists under:", LIST_DIR)

Generated split lists under: /content/yolo_dataset/splits


In [ ]:
# =========================
# 6) Per-fold AUG（Tail Copy-Paste）
# =========================

In [ ]:
AUG_ROOT = "/content/yolo_dataset_aug"
shutil.rmtree(AUG_ROOT, ignore_errors=True)
os.makedirs(AUG_ROOT, exist_ok=True)

In [ ]:
# 蒐集 tail instances
TAIL_IDS = [1,2,3]  # hov/person/motorcycle
instances = []
def _read_img_hw(p):
    im = cv2.imread(p, cv2.IMREAD_COLOR)
    return (im, im.shape[:2]) if im is not None else (None, None)

print("Collecting tail instances...")
for lab_p in tqdm(glob.glob(os.path.join(LAB_DIR, "*.txt"))):
    k = os.path.splitext(os.path.basename(lab_p))[0]
    img_p = os.path.join(IMG_DIR, f"{k}.png")
    im, hw = _read_img_hw(img_p)
    if im is None: continue
    H, W = hw
    with open(lab_p, "r") as f:
        for line in f:
            p=line.strip().split()
            if len(p)!=5: continue
            c=int(p[0]); cx,cy,ww,hh = map(float, p[1:])
            if c not in TAIL_IDS: continue
            x = int((cx - ww/2)*W); y = int((cy - hh/2)*H)
            w = max(1, int(ww*W)); h = max(1, int(hh*H))
            x = np.clip(x, 0, max(0,W-1)); y = np.clip(y, 0, max(0,H-1))
            if x+w>W: w = W-x
            if y+h>H: h = H-y
            if w<4 or h<4: continue
            crop = im[y:y+h, x:x+w].copy()
            if crop.size>0: instances.append((c, crop))
print("Tail instances collected:", len(instances))

100%|██████████| 950/950 [00:46<00:00, 20.25it/s]

Tail instances collected: 9971


In [ ]:
def _ok_place(px, py, nw, nh, W, H):
    if py < int(0.35*H) or (py+nh) > int(0.95*H):
        return False
    if px < 2 or py < 2 or (px+nw) > (W-2) or (py+nh) > (H-2):
        return False
    return True

def _iou_xyxy(a, b):
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    ix1, iy1 = max(ax1, bx1), max(ay1, by1)
    ix2, iy2 = min(ax2, bx2), min(ay2, by2)
    iw, ih = max(0, ix2-ix1), max(0, iy2-iy1)
    inter = iw * ih
    if inter <= 0: return 0.0
    area_a = max(0, ax2-ax1) * max(0, ay2-ay1)
    area_b = max(0, bx2-ax1) * max(0, by2-ay1)
    return inter / (area_a + area_b - inter + 1e-9)

def _boxes_from_lab(lab_path, W, H):
    boxes=[]
    if os.path.isfile(lab_path):
        with open(lab_path, "r") as f:
            for line in f:
                p=line.strip().split()
                if len(p)!=5: continue
                c=int(p[0]); cx,cy,ww,hh = map(float, p[1:])
                x1 = (cx - ww/2)*W; y1 = (cy - hh/2)*H
                x2 = (cx + ww/2)*W; y2 = (cy + hh/2)*H
                boxes.append([c, x1,y1,x2,y2])
    return boxes

def _random_light_jitter(img):
    img = img.astype(np.float32)
    alpha = np.random.uniform(0.9, 1.1)
    beta  = np.random.uniform(-8, 8)
    out = np.clip(img*alpha + beta, 0, 255).astype(np.uint8)
    return out

def _random_motion_blur(img, k=3):
    if random.random()<0.5: return img
    kernel = np.zeros((k,k), dtype=np.float32)
    if random.random()<0.5: kernel[k//2, :] = 1.0
    else: kernel[:, k//2] = 1.0
    kernel /= kernel.sum()
    return cv2.filter2D(img, -1, kernel)

In [ ]:
AUG_PER_IMAGE_BASE = 1
AUG_PER_IMAGE_IF_NO_TAIL = 2
PERSON_SMALL_SCALE = (0.5, 0.9)
OTHERS_SCALE       = (0.7, 1.1)
IOU_BLOCK_THR = 0.3

In [ ]:
for fi, fold in enumerate(folds):
    fold_root = os.path.join(AUG_ROOT, f"f{fi}")
    IMG_OUT = os.path.join(fold_root, "images")
    LAB_OUT = os.path.join(fold_root, "labels")
    os.makedirs(IMG_OUT, exist_ok=True); os.makedirs(LAB_OUT, exist_ok=True)

    train_keys = set(fold["train"])
    train_imgs = [os.path.join(IMG_DIR, f"{k}.png") for k in train_keys if os.path.isfile(os.path.join(IMG_DIR, f"{k}.png"))]
    print(f"\n[Fold {fi}] train images for AUG:", len(train_imgs))

    aug_cnt = 0
    for img_p in tqdm(train_imgs, desc=f"Fold{fi} AUG"):
        k = os.path.splitext(os.path.basename(img_p))[0]
        lab_p = os.path.join(LAB_DIR, f"{k}.txt")
        bg = cv2.imread(img_p, cv2.IMREAD_COLOR)
        if bg is None: continue
        H, W = bg.shape[:2]
        boxes = _boxes_from_lab(lab_p, W, H)
        vec = presence_vector.get(k, [0]*len(CLASSES))
        n_add = AUG_PER_IMAGE_IF_NO_TAIL if ((vec[1]==0) or (vec[2]==0) or (vec[3]==0)) else AUG_PER_IMAGE_BASE

        pasted_xyxy=[]; pasted_yolo=[]
        for _ in range(n_add):
            if not instances: break
            c, crop = random.choice(instances)
            scale_min, scale_max = PERSON_SMALL_SCALE if c==2 else OTHERS_SCALE
            scale = np.random.uniform(scale_min, scale_max)
            nh, nw = max(4, int(crop.shape[0]*scale)), max(4, int(crop.shape[1]*scale))
            patch = cv2.resize(crop, (nw, nh), interpolation=cv2.INTER_LINEAR)
            patch = _random_light_jitter(_random_motion_blur(patch, k=3))
            ok=False
            for _try in range(10):
                px = random.randint(0, max(1, W - nw)); py = random.randint(0, max(1, H - nh))
                if not _ok_place(px, py, nw, nh, W, H): continue
                x1,y1,x2,y2 = px,py, px+nw, py+nh
                bad=False
                for _, bx1,by1,bx2,by2 in boxes:
                    if _iou_xyxy([x1,y1,x2,y2],[bx1,by1,bx2,by2])>IOU_BLOCK_THR: bad=True; break
                if bad: continue
                for bx1,by1,bx2,by2 in pasted_xyxy:
                    if _iou_xyxy([x1,y1,x2,y2],[bx1,by1,bx2,by2])>IOU_BLOCK_THR: bad=True; break
                if bad: continue
                ok=True; break
            if not ok: continue
            bg[py:py+nh, px:px+nw] = patch
            pasted_xyxy.append([x1,y1,x2,y2])
            cx = (x1+x2)/2.0/W; cy=(y1+y2)/2.0/H; ww=(x2-x1)/W; hh=(y2-y1)/H
            pasted_yolo.append([c, cx, cy, ww, hh])

        if not pasted_yolo: continue
        new_key = f"{k}_aug_f{fi}"
        out_img = os.path.join(IMG_OUT, f"{new_key}.png")
        out_lab = os.path.join(LAB_OUT, f"{new_key}.txt")
        cv2.imwrite(out_img, bg)

        yolo_boxes = []
        if os.path.isfile(lab_p):
            with open(lab_p, "r") as f:
                for line in f:
                    line=line.strip()
                    if not line: continue
                    yolo_boxes.append(line)
        for c,cx,cy,ww,hh in pasted_yolo:
            yolo_boxes.append(f"{c} {cx:.6f} {cy:.6f} {ww:.6f} {hh:.6f}")
        with open(out_lab, "w") as f:
            f.write("\n".join(yolo_boxes))
        aug_cnt += 1
    print(f"[Fold {fi}] augmented images:", aug_cnt)


[Fold 0] train images for AUG: 696


Fold0 AUG: 100%|██████████| 696/696 [01:50<00:00,  6.32it/s]


[Fold 0] augmented images: 696

[Fold 1] train images for AUG: 700


Fold1 AUG: 100%|██████████| 700/700 [01:50<00:00,  6.32it/s]


[Fold 1] augmented images: 700

[Fold 2] train images for AUG: 697


...

[Fold 2] augmented images: 697

[Fold 3] train images for AUG: 691


Fold3 AUG: 100%|██████████| 691/691 [01:50<00:00,  6.24it/s]


[Fold 3] augmented images: 691

[Fold 4] train images for AUG: 697


Fold4 AUG: 100%|██████████| 697/697 [01:52<00:00,  6.21it/s]

[Fold 4] augmented images: 697


In [ ]:
# 重寫 train_final/val_final（每折只加本折的 AUG）
for fi, fold in enumerate(folds):
    base_rfs = os.path.join(LIST_DIR, f"fold{fi}_train_rfs.txt")
    val_txt  = os.path.join(LIST_DIR, f"fold{fi}_val.txt")
    with open(base_rfs) as f:
        rfs_paths = [ln.strip() for ln in f if ln.strip()]
    aug_paths = sorted(glob.glob(os.path.join(AUG_ROOT, f"f{fi}", "images", "*.png")))
    train_final = os.path.join(LIST_DIR, f"fold{fi}_train_final.txt")
    val_final   = os.path.join(LIST_DIR, f"fold{fi}_val_final.txt")
    with open(train_final, "w") as f:
        for p in rfs_paths: f.write(p+"\n")
        for p in aug_paths: f.write(p+"\n")
    with open(val_final, "w") as f:
        with open(val_txt) as vf:
            for ln in vf:
                if ln.strip(): f.write(ln)
    print(f"[Fold {fi}] train_final: {len(rfs_paths)} + aug {len(aug_paths)} = {len(rfs_paths)+len(aug_paths)}")
print("✅ per-fold AUG 完成（無洩漏）")


[Fold 0] train_final: 696 + aug 696 = 1392
[Fold 1] train_final: 700 + aug 700 = 1400
[Fold 2] train_final: 697 + aug 697 = 1394
[Fold 3] train_final: 691 + aug 691 = 1382
[Fold 4] train_final: 697 + aug 697 = 1394
✅ per-fold AUG 完成（無洩漏）


# RT-DETR 用 data.yaml

In [ ]:
YAML_DIR = os.path.join(YOLO_ROOT, "yamls")
os.makedirs(YAML_DIR, exist_ok=True)
for fi in range(5):
    yaml_path = os.path.join(YAML_DIR, f"fold{fi}.yaml")
    with open(yaml_path, "w") as f:
        f.write(f"""# auto-generated data.yaml for fold {fi}
path: {YOLO_ROOT}
train: {os.path.join(YOLO_ROOT, "splits", f"fold{fi}_train_final.txt")}
val: {os.path.join(YOLO_ROOT, "splits", f"fold{fi}_val_final.txt")}
nc: 4
names: ["car","hov","person","motorcycle"]
""")
print("Wrote YOLO/RT-DETR data yamls under:", YAML_DIR)

Wrote YOLO/RT-DETR data yamls under: /content/yolo_dataset/yamls


In [ ]:
!pip -q install ultralytics==8.3.204

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.2 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO, RTDETR

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# 先只訓練一折
FOLD_TO_RUN = 0
DEVICE = 0

# VRAM 安全參數
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
try: torch.set_float32_matmul_precision('high')
except: pass
try: torch.cuda.set_per_process_memory_fraction(0.90, device=DEVICE)
except: pass

IMG_SZ = 768
BATCH  = 8
EPOCHS = 150
LR0    = 1e-3
WD     = 5e-4

PROJECT = "/content/runs_rtdetr_onefold"
os.makedirs(PROJECT, exist_ok=True)
run_name = f"rtdetr_l_f{FOLD_TO_RUN}_{int(time.time())}"


In [ ]:
# 使用前面建立好的 data.yaml
assert 'YAML_DIR' in globals(), "請先執行前面的資料準備，建立 YAML_DIR"
rtd_yaml = os.path.join(YAML_DIR, f"fold{FOLD_TO_RUN}.yaml")


In [ ]:
print(f"\n========== 🚀 RT-DETR-L Fold {FOLD_TO_RUN} 訓練開始：{run_name} ==========")
model = RTDETR('rtdetr-l.pt')

model.train(
    data=rtd_yaml,
    epochs=EPOCHS,
    imgsz=IMG_SZ,
    device=DEVICE,
    batch=BATCH,
    workers=2,
    optimizer='AdamW',
    lr0=LR0,
    weight_decay=WD,

    # 小物/長尾友善增強
    mosaic=0.30,
    mixup=0.05,
    copy_paste=0.20,
    hsv_s=0.40,
    degrees=5.0,
    scale=0.10,
    translate=0.05,
    shear=0.0,

    # 其它通用訓練參數
    rect=False,
    close_mosaic=20,
    cos_lr=True,
    patience=25,
    amp=True,              # 自動混合精度省顯存

    # 從零初始化!!
    pretrained=False,

    project=PROJECT,
    name=run_name,
    verbose=True,
    save_period=10
)



========== 🚀 RT-DETR-L Fold 0 訓練開始：rtdetr_l_f0_1761466324 ==========
New https://pypi.org/project/ultralytics/8.3.221 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.204 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=20, cls=0.5, compile=False, conf=None, copy_paste=0.2, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/yolo_dataset/yamls/fold0.yaml, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.4, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.05, mode=train, model=rtdetr-l.pt, momentum=

/usr/local/lib/python3.12/dist-packages/torch/autograd/graph.py:829: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:93.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      1/150      9.56G      1.791     0.3541     0.4304        250        768: 100% ━━━━━━━━━━━━ 174/174 1.1it/s 2:35
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 12/12 0.9it/s 14.0s
                   all        190       6429     0.0305      0.139     0.0348    0.00907

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


/usr/local/lib/python3.12/dist-packages/torch/autograd/graph.py:829: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:93.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


      2/150        11G      1.336     0.4898     0.1939        410        768: 100% ━━━━━━━━━━━━ 174/174 1.3it/s 2:09
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 12/12 2.4it/s 4.9s
                   all        190       6429      0.539      0.187     0.0567     0.0199

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
      3/150      11.9G      1.021     0.6548      0.117        287        768: 0% ──────────── 0/174  0.7s

...

/usr/local/lib/python3.12/dist-packages/torch/autograd/graph.py:829: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:93.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


     81/150      12.9G     0.6501     0.4491    0.05558        283        768: 100% ━━━━━━━━━━━━ 174/174 1.4it/s 2:08
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 12/12 2.4it/s 5.0s
                   all        190       6429      0.689       0.68      0.656      0.225

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size
     82/150      12.2G     0.6578     0.4395    0.05354        438        768: 0% ──────────── 0/174  0.7s

/usr/local/lib/python3.12/dist-packages/torch/autograd/graph.py:829: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:93.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


     82/150        13G     0.6402     0.4496    0.05381        403        768: 100% ━━━━━━━━━━━━ 174/174 1.4it/s 2:08
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 12/12 2.4it/s 5.0s
                   all        190       6429      0.684      0.679      0.661      0.226
EarlyStopping: Training stopped early as no improvement observed in last 25 epochs. Best results observed at epoch 57, best model saved as best.pt.
To update EarlyStopping(patience=25) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

82 epochs completed in 3.069 hours.
Optimizer stripped from /content/runs_rtdetr_onefold/rtdetr_l_f0_1761466324/weights/last.pt, 66.2MB
Optimizer stripped from /content/runs_rtdetr_onefold/rtdetr_l_f0_1761466324/weights/best.pt, 66.2MB

Validating /content/runs_rtdetr_onefold/rtdetr_l_f0_1761466324/weights/best.pt...
Ultralytics 8.3.204 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ef3a81ae330>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

In [ ]:
run_dir = os.path.join(PROJECT, run_name)
best_pt = os.path.join(run_dir, "weights", "best.pt")
assert os.path.isfile(best_pt), f"❌ 未找到 best.pt：{best_pt}"

In [ ]:
print(f"🔎 驗證 fold{FOLD_TO_RUN}（best.pt）...")
m_best = RTDETR(best_pt)
val_res = m_best.val(
    data=rtd_yaml,
    imgsz=IMG_SZ,
    device=DEVICE,
    iou=0.5,       # 與 mAP50 對齊
    conf=0.001,
    split='val',
    plots=True
)
metrics = getattr(val_res, "results_dict", {})
print(f"📊 fold{FOLD_TO_RUN} metrics:", metrics)

🔎 驗證 fold0（best.pt）...
Ultralytics 8.3.204 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
rt-detr-l summary: 302 layers, 31,991,960 parameters, 0 gradients, 103.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4018.4±125.4 MB/s, size: 4079.2 KB)
val: Scanning /content/yolo_dataset/labels.cache... 190 images, 1 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 190/190 383.1Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 12/12 0.5it/s 26.3s
                   all        190       6429      0.671      0.679       0.67      0.234
                   car        186       4588      0.832      0.881      0.882      0.346
                   hov        108        237       0.75      0.831      0.836      0.318
                person        100        581      0.524      0.428       0.41      0.106
            motorcycle         95       1023      0.579      0.577      0.554      0.164
Speed: 1.6ms preprocess

In [ ]:
# 備份到 Drive
DRIVE_SAVE_DIR = "/content/drive/MyDrive/ComputerVision_DL/hw2_longtail_od/cv_hw2_runs_rtdetr"
os.makedirs(DRIVE_SAVE_DIR, exist_ok=True)
drive_best = os.path.join(DRIVE_SAVE_DIR, f"rtdetr_l_f{FOLD_TO_RUN}_best.pt")
shutil.copy2(best_pt, drive_best)
for fn in ["results.csv", "args.yaml", "results.png", "PR_curve.png"]:
    fp = os.path.join(run_dir, fn)
    if os.path.isfile(fp):
        shutil.copy2(fp, os.path.join(DRIVE_SAVE_DIR, f"rtdetr_l_f{FOLD_TO_RUN}_{fn}"))
print("✅ 已備份到 Drive：", drive_best)

✅ 已備份到 Drive： /content/drive/MyDrive/ComputerVision_DL/hw2_longtail_od/cv_hw2_runs_rtdetr/rtdetr_l_f0_best.pt


# 推論

In [ ]:
# ============================================================
# 三折YOLOv8-p +單折RT-DETR-R18 WBF +VAL 掃參數 +TEST
# ============================================================

import os, glob, csv, cv2, time, json
import numpy as np

In [ ]:
WEIGHT_DIR    = "/content/drive/MyDrive/ComputerVision_DL/hw2_longtail_od/ckpts"
YOLO_WEIGHTS  = [
    os.path.join(WEIGHT_DIR, "best_f0.pt"),
    os.path.join(WEIGHT_DIR, "best_f1.pt"),
    os.path.join(WEIGHT_DIR, "best_f2.pt"),
]
RTDETR_WEIGHT = os.path.join(WEIGHT_DIR, "best_rtdetr_f0.pt")

OUT_DIR       = "/content/drive/MyDrive/ComputerVision_DL/hw2_longtail_od/submissions"
os.makedirs(OUT_DIR, exist_ok=True)

In [ ]:
assert "TEST_DIR" in globals(), "請先在上方定義 TEST_DIR = os.path.join(DATASET_ROOT, 'test')"
assert os.path.isdir(TEST_DIR), f"TEST_DIR 不存在：{TEST_DIR}"


In [ ]:
# 推論/融合參數
DEVICE       = 0
GLOBAL_IOU   = 0.55
SCALES       = [672, 704, 768]
HFLIPS       = [False, True]
BASE_CONF    = 0.001
MAX_DET      = 3000
SCORE_POWER  = 1.0
WBF_IOU      = 0.50
VAR_LAMBDA   = 1.2
VAR_SCALE    = 8.0

# 類別門檻：car, hov, person, motorcycle
PER_CLASS_CONF = {0: 0.34, 1: 0.32, 2: 0.30, 3: 0.30}

# 模型校準
T_BY_MODEL = {"yolo": 1.0, "rtdetr": 1.4}
B_BY_MODEL_CLASS = {
    "yolo":   {0:0.0, 1:0.0, 2:0.0, 3:0.0},
    "rtdetr": {0:-0.05, 1:-0.05, 2:-0.15, 3:-0.10},
}
W_BY_MODEL_CLASS = {
    "yolo":   {0:1.0, 1:1.0, 2:1.0, 3:1.0},
    "rtdetr": {0:0.8, 1:0.7, 2:0.5, 3:0.6},
}
ASSIST_ONLY_MODELS = {"rtdetr"}

USE_SOFTNMS   = True
SOFTNMS_IOU_P = 0.55
SOFTNMS_IOU_O = 0.50
SOFTNMS_SIGMA = 0.5
SOFTNMS_MIN_S = 0.05


In [ ]:
# 小工具
def clipf(v, lo, hi):
    return max(lo, min(hi, v))

def iou_xyxy(a, b):
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    ix1, iy1 = max(ax1, bx1), max(ay1, by1)
    ix2, iy2 = min(ax2, bx2), min(ay2, by2)
    iw, ih = max(0.0, ix2 - ix1), max(0.0, iy2 - iy1)
    inter = iw * ih
    if inter <= 0.0: return 0.0
    area_a = max(0.0, ax2 - ax1) * max(0.0, ay2 - ay1)
    area_b = max(0.0, bx2 - bx1) * max(0.0, by2 - by1)
    return inter / (area_a + area_b - inter + 1e-9)

def classwise_soft_nms(xyxy_list, score_list, iou_thr, sigma=SOFTNMS_SIGMA, min_score=SOFTNMS_MIN_S):
    boxes  = np.array(xyxy_list, float).copy()
    scores = np.array(score_list, float).copy()
    if boxes.size == 0:
        return [], []
    keep_boxes, keep_scores = [], []
    idxs = np.arange(boxes.shape[0])
    while idxs.size > 0:
        i = np.argmax(scores[idxs])
        sel = idxs[i]
        max_box   = boxes[sel].copy()
        max_score = float(scores[sel])
        keep_boxes.append(max_box.tolist())
        keep_scores.append(max_score)

        remain = np.delete(idxs, i)
        if remain.size == 0: break
        ious = np.array([iou_xyxy(max_box, boxes[j]) for j in remain], float)
        decay = np.exp(-(ious * ious) / max(1e-9, sigma))
        scores[remain] = np.where(ious > iou_thr, scores[remain] * decay, scores[remain])
        idxs = remain[scores[remain] >= min_score]
    return keep_boxes, keep_scores

def wbf_with_model_calibration(
    boxes, scores, src_models, src_scales, classes,
    iou_thr, score_power, img_w, img_h,
    T_by_model, B_by_model_class, W_by_model_class,
    agree_gamma_models=0.30, agree_gamma_scales=0.15,
    var_lambda=1.2, VAR_SCALE=8.0, assist_only_models=None, eps=1e-6
):
    """
    帶模型校準/權重的 WBF + logit 分數融合（保證分數在(0,1)）
    - 位置加權: (p_calib ** score_power) * W_{model,class}
    - 分數融合: logit(base) + γM*ln(#models) + γS*ln(#scales) - λ*VAR_SCALE*var
    - Assist-only: 群組全是 assist 模型則跳過
    """
    if len(boxes)==0: return [], []

    # per-box 校準（分數）與位置權重
    adj_scores, pos_weights = [], []
    for p, m, c in zip(scores, src_models, classes):
        p = float(np.clip(p, eps, 1.0 - eps))
        Tm = T_by_model.get(m, 1.0)
        Bmc= B_by_model_class.get(m, {}).get(c, 0.0)
        logit = np.log(p/(1.0 - p))
        pm = 1.0 / (1.0 + np.exp(-(logit / Tm + Bmc)))
        adj_scores.append(pm)
        Wmc= W_by_model_class.get(m, {}).get(c, 1.0)
        pos_weights.append((pm ** score_power) * Wmc)

    boxes = boxes.copy()
    adj_scores = np.array(adj_scores, float)
    pos_weights= np.array(pos_weights, float)

    used = [False] * len(boxes)
    fused_boxes, fused_scores = [], []
    order = np.argsort(adj_scores)[::-1]

    for i in order:
        if used[i]: continue
        group = [i]; used[i] = True
        base = boxes[i]
        # 建群
        for j in order:
            if used[j]: continue
            if iou_xyxy(base, boxes[j]) >= iou_thr:
                group.append(j); used[j] = True

        g_boxes  = np.array([boxes[k] for k in group], float)
        g_p      = adj_scores[group]
        g_wpos   = pos_weights[group]
        g_models = [src_models[k] for k in group]
        g_scales = [src_scales[k] for k in group]

        # Assist-only：群組若全是 assist 模型則跳過
        if assist_only_models and all([(m in assist_only_models) for m in g_models]):
            continue

        # 位置融合
        w = g_wpos.reshape(-1, 1)
        fused = (g_boxes * w).sum(0) / (w.sum(0) + 1e-9)

        # 分數融合（logit）
        base_score = float(g_p.max())
        base_logit = np.log(base_score/(1.0 - base_score))
        uniq_models = len(set(g_models))
        uniq_scales = len(set(g_scales))
        agree_term  = agree_gamma_models * np.log(max(1, uniq_models)) \
                    + agree_gamma_scales * np.log(max(1, uniq_scales))
        norm = np.array([img_w, img_h, img_w, img_h], float).reshape(1,4)
        var_mean = float(np.var(g_boxes / norm, axis=0).mean())
        var_term = var_lambda * VAR_SCALE * var_mean
        final_logit = base_logit + agree_term - var_term
        final_score = 1.0 / (1.0 + np.exp(-final_logit))  # ∈ (0,1)

        fused_boxes.append(fused.tolist())
        fused_scores.append(float(final_score))

    return fused_boxes, fused_scores

In [ ]:
# 載入模型
YOLO_WEIGHTS = [w for w in YOLO_WEIGHTS if os.path.isfile(w)]
assert YOLO_WEIGHTS, f"找不到 YOLO 權重：{YOLO_WEIGHTS}"
assert os.path.isfile(RTDETR_WEIGHT), f"找不到 RT-DETR 權重：{RTDETR_WEIGHT}"
print("YOLO 權重：", YOLO_WEIGHTS)
print("RT-DETR ：", RTDETR_WEIGHT)

yolo_models  = [YOLO(w) for w in YOLO_WEIGHTS]
rtdetr_model = YOLO(RTDETR_WEIGHT)

YOLO 權重： ['/content/drive/MyDrive/ComputerVision_DL/hw2_longtail_od/ckpts/best_f0.pt', '/content/drive/MyDrive/ComputerVision_DL/hw2_longtail_od/ckpts/best_f1.pt', '/content/drive/MyDrive/ComputerVision_DL/hw2_longtail_od/ckpts/best_f2.pt']
RT-DETR ： /content/drive/MyDrive/ComputerVision_DL/hw2_longtail_od/ckpts/best_rtdetr_f0.pt


In [ ]:
# 掃參數

# ---- 用 splits 產生 VAL 清單 ----
SPLIT_DIR   = "/content/yolo_dataset/splits"
FOLDS_JSON  = "/content/reports/folds_5.json"
TRAIN_FOLDS = [0, 1, 2]
ALL_FOLDS   = list(range(5))
VAL_FOLDS   = [f for f in ALL_FOLDS if f not in TRAIN_FOLDS]

def _find_val_list(split_dir, fi):
    cand = []
    patts = [
        f"val_fold{fi}.txt", f"fold{fi}_val.txt", f"val_f{fi}.txt",
        f"fold{fi}.val.txt", f"val{fi}.txt"
    ]
    for p in patts:
        fp = os.path.join(split_dir, p)
        if os.path.isfile(fp):
            cand.append(fp)
    if not cand:
        for fp in glob.glob(os.path.join(split_dir, "*.txt")):
            name = os.path.basename(fp).lower()
            if f"{fi}" in name and "val" in name:
                cand.append(fp)
    return cand[0] if cand else None

def _read_img_label_pairs(list_path):
    imgs, labels = [], []
    with open(list_path, "r") as rf:
        for line in rf:
            imgp = line.strip()
            if not imgp:
                continue
            if not os.path.isabs(imgp):
                imgp = os.path.abspath(imgp)
            imgs.append(imgp)
            lp = imgp.replace("/images/", "/labels/").replace("\\images\\", "\\labels\\")
            lp = os.path.splitext(lp)[0] + ".txt"
            labels.append(lp)
    return imgs, labels

VAL_IMAGE_LIST, VAL_LABEL_LIST = [], []
for fi in VAL_FOLDS:
    lst = _find_val_list(SPLIT_DIR, fi)
    assert lst is not None, f"在 {SPLIT_DIR} 找不到 fold {fi} 的 val 清單檔。"
    imgs, labs = _read_img_label_pairs(lst)
    VAL_IMAGE_LIST += imgs
    VAL_LABEL_LIST += labs

print(f"[VAL] folds = {VAL_FOLDS} → 影像 {len(VAL_IMAGE_LIST)} 張")


[VAL] folds = [3, 4] → 影像 382 張


In [ ]:
# 準備VAL 的GT
def load_yolo_labels(txt_path, img_w, img_h):
    gts = []
    if not os.path.isfile(txt_path):
        return gts
    with open(txt_path, "r") as rf:
        for line in rf:
            ss = line.strip().split()
            if len(ss) < 5:
                continue
            c = int(ss[0]); x=float(ss[1]); y=float(ss[2]); w=float(ss[3]); h=float(ss[4])
            x1 = (x - w/2) * img_w; y1 = (y - h/2) * img_h
            x2 = (x + w/2) * img_w; y2 = (y + h/2) * img_h
            gts.append((c, x1, y1, x2, y2))
    return gts

gt_by_img = {}
val_imgs  = VAL_IMAGE_LIST
for imgp, labp in zip(VAL_IMAGE_LIST, VAL_LABEL_LIST):
    im = cv2.imread(imgp); H, W = im.shape[:2]
    stem = os.path.splitext(os.path.basename(imgp))[0]
    gt_by_img[stem] = load_yolo_labels(labp, W, H)

In [ ]:
# 簡版AP50計算
def compute_ap50(gt_by_img, pred_by_img):
    preds = []
    for img_id, items in pred_by_img.items():
        for c, x1,y1,x2,y2, s in items:
            preds.append((s, img_id, c, x1,y1,x2,y2))
    preds.sort(key=lambda x: -x[0])

    tp = []; fp = []
    matched = {img_id: [False]*len(gt_by_img.get(img_id, [])) for img_id in gt_by_img}

    def _iou(a, b):
        ax1,ay1,ax2,ay2 = a; bx1,by1,bx2,by2 = b
        ix1,iy1 = max(ax1,bx1), max(ay1,by1)
        ix2,iy2 = min(ax2,bx2), min(ay2,by2)
        iw,ih = max(0,ix2-ix1), max(0,iy2-iy1)
        inter = iw*ih
        if inter<=0: return 0.0
        area_a = max(0,ax2-ax1)*max(0,ay2-ay1)
        area_b = max(0,bx2-bx1)*max(0,by2-by1)
        return inter/(area_a+area_b-inter+1e-9)

    GT_POS = sum([len(v) for v in gt_by_img.values()])
    for s, img_id, c, x1,y1,x2,y2 in preds:
        best_iou = 0.0; best_j = -1
        gts = gt_by_img.get(img_id, [])
        for j, (gc, gx1,gy1,gx2,gy2) in enumerate(gts):
            if gc != c or matched[img_id][j]:
                continue
            i = _iou((x1,y1,x2,y2), (gx1,gy1,gx2,gy2))
            if i > best_iou:
                best_iou = i; best_j = j
        if best_iou >= 0.5 and best_j >= 0:
            matched[img_id][best_j] = True
            tp.append(1); fp.append(0)
        else:
            tp.append(0); fp.append(1)

    if not tp:
        return 0.0
    tp = np.cumsum(tp); fp = np.cumsum(fp)
    recall = tp / max(1, GT_POS)
    precision = tp / np.maximum(1, tp + fp)
    ap = 0.0
    for t in np.linspace(0,1,11):
        p = precision[recall >= t].max() if np.any(recall >= t) else 0.0
        ap += p / 11.0
    return float(ap)

In [ ]:
# 預測
def predict_one_image(im_path, params):
    im = cv2.imread(im_path); H, W = im.shape[:2]
    all_xyxy, all_scores, all_cls = [], [], []
    all_models, all_scales = [], []

    for sz in params["SCALES"]:
        for flip in [False, True]:
            im_in = cv2.flip(im, 1) if flip else im
            # YOLO
            for mdl in params["YOLO_MODELS"]:
                r = mdl.predict(source=im_in, imgsz=sz, conf=params["BASE_CONF"], iou=params["GLOBAL_IOU"],
                                device=params["DEVICE"], verbose=False, max_det=params["MAX_DET"], augment=False)[0]
                if r and r.boxes is not None:
                    xyxy = r.boxes.xyxy.cpu().numpy()
                    conf = r.boxes.conf.cpu().numpy()
                    cls  = r.boxes.cls.cpu().numpy().astype(int)
                    if flip and xyxy.size:
                        x1 = xyxy[:,0].copy(); x2 = xyxy[:,2].copy()
                        xyxy[:,0] = W - x2;    xyxy[:,2] = W - x1
                    if xyxy.size:
                        all_xyxy.append(xyxy); all_scores.append(conf); all_cls.append(cls)
                        all_models.append(np.array(["yolo"]*xyxy.shape[0], object))
                        all_scales.append(np.array([f"s{sz}"]*xyxy.shape[0], object))
            # RT-DETR
            r = params["RT_MODEL"].predict(source=im_in, imgsz=sz, conf=params["BASE_CONF"], iou=params["GLOBAL_IOU"],
                                           device=params["DEVICE"], verbose=False, max_det=params["MAX_DET"], augment=False)[0]
            if r and r.boxes is not None:
                xyxy = r.boxes.xyxy.cpu().numpy()
                conf = r.boxes.conf.cpu().numpy()
                cls  = r.boxes.cls.cpu().numpy().astype(int)
                if flip and xyxy.size:
                    x1 = xyxy[:,0].copy(); x2 = xyxy[:,2].copy()
                    xyxy[:,0] = W - x2;    xyxy[:,2] = W - x1
                if xyxy.size:
                    all_xyxy.append(xyxy); all_scores.append(conf); all_cls.append(cls)
                    all_models.append(np.array(["rtdetr"]*xyxy.shape[0], object))
                    all_scales.append(np.array([f"s{sz}"]*xyxy.shape[0], object))

    if not all_xyxy:
        return []

    all_xyxy   = np.concatenate(all_xyxy, axis=0)
    all_scores = np.concatenate(all_scores, axis=0)
    all_cls    = np.concatenate(all_cls, axis=0)
    all_models = np.concatenate(all_models, axis=0)
    all_scales = np.concatenate(all_scales, axis=0)

    out = []
    for c in (0,1,2,3):
        idxs = np.where(all_cls == c)[0]
        if idxs.size == 0:
            continue

        b_list = all_xyxy[idxs].tolist()
        s_list = all_scores[idxs].tolist()
        m_list = all_models[idxs].tolist()
        sca_list = all_scales[idxs].tolist()
        cls_list = [c] * len(idxs)

        group_iou = params["WBF_IOU_PERSON"] if c == 2 else params["WBF_IOU"]
        fused_b, fused_s = wbf_with_model_calibration(
            b_list, s_list, m_list, sca_list, cls_list,
            iou_thr=group_iou, score_power=params["SCORE_POWER"], img_w=W, img_h=H,
            T_by_model=params["T_BY_MODEL"], B_by_model_class=params["B_BY_MODEL_CLASS"],
            W_by_model_class=params["W_BY_MODEL_CLASS"],
            agree_gamma_models=params["AGREE_GAMMA_M"], agree_gamma_scales=params["AGREE_GAMMA_S"],
            var_lambda=params["VAR_LAMBDA"], VAR_SCALE=params["VAR_SCALE"],
            assist_only_models=params["ASSIST_ONLY_MODELS"]
        )

        # Soft-NMS
        post_iou = params["SOFTNMS_IOU_P"] if c == 2 else params["SOFTNMS_IOU_O"]
        if params["USE_SOFTNMS"] and fused_b:
            fused_b, fused_s = classwise_soft_nms(fused_b, fused_s, iou_thr=post_iou,
                                                  sigma=params["SOFTNMS_SIGMA"], min_score=0.05)
        thr = params["PER_CLASS_CONF"].get(c, 0.32)
        for bb, sc in zip(fused_b, fused_s):
            if sc < thr:
                continue
            x1,y1,x2,y2 = bb
            out.append((c, x1,y1,x2,y2, float(sc)))
    return out

In [ ]:
# 掃描空間
param_space = [
    {"T_rtd":t, "B_rtd_p":b, "W_rtd_p":w, "WBF_p":wbf, "VL":vl}
    for t   in [1.3, 1.4, 1.5]
    for b   in [-0.25, -0.15, -0.10]
    for w   in [0.3, 0.5]
    for wbf in [0.55, 0.60]
    for vl  in [1.0, 1.2]
]

BASE_PARAMS = {
    "SCALES": SCALES, "GLOBAL_IOU": GLOBAL_IOU, "BASE_CONF": BASE_CONF, "MAX_DET": MAX_DET,
    "DEVICE": DEVICE, "YOLO_MODELS": yolo_models, "RT_MODEL": rtdetr_model,
    "SCORE_POWER": SCORE_POWER, "WBF_IOU": WBF_IOU, "WBF_IOU_PERSON": 0.60,
    "AGREE_GAMMA_M": 0.30, "AGREE_GAMMA_S": 0.15, "VAR_LAMBDA": VAR_LAMBDA, "VAR_SCALE": VAR_SCALE,
    "T_BY_MODEL": T_BY_MODEL.copy(),
    "B_BY_MODEL_CLASS": {k:v.copy() for k,v in B_BY_MODEL_CLASS.items()},
    "W_BY_MODEL_CLASS": {k:v.copy() for k,v in W_BY_MODEL_CLASS.items()},
    "ASSIST_ONLY_MODELS": ASSIST_ONLY_MODELS.copy(),
    "PER_CLASS_CONF": PER_CLASS_CONF.copy(),
    "SOFTNMS_IOU_P": SOFTNMS_IOU_P, "SOFTNMS_IOU_O": SOFTNMS_IOU_O,
    "SOFTNMS_SIGMA": SOFTNMS_SIGMA, "USE_SOFTNMS": USE_SOFTNMS,
}

In [ ]:
val_subset = val_imgs[:400]  # 加速
results = []

early_stop_patience = 8
best_ap = 0
stable_count = 0

for i, cfg in enumerate(param_space):
    params = BASE_PARAMS.copy()
    params["T_BY_MODEL"] = {"yolo":1.0, "rtdetr":cfg["T_rtd"]}
    params["B_BY_MODEL_CLASS"] = {
        "yolo":   {0:0,1:0,2:0,3:0},
        "rtdetr": {0:-0.05,1:-0.05,2:cfg["B_rtd_p"],3:-0.10}
    }
    params["W_BY_MODEL_CLASS"] = {
        "yolo":   {0:1,1:1,2:1,3:1},
        "rtdetr": {0:0.8,1:0.7,2:cfg["W_rtd_p"],3:0.6}
    }
    params["WBF_IOU_PERSON"] = cfg["WBF_p"]
    params["VAR_LAMBDA"]     = cfg["VL"]

    pred_by_img = {}
    for pth in val_subset:
        stem = os.path.splitext(os.path.basename(pth))[0]
        preds = predict_one_image(pth, params)
        pred_by_img[stem] = preds

    ap50 = compute_ap50(gt_by_img, pred_by_img)
    results.append((ap50, cfg))
    print(f"[VAL] AP50={ap50:.5f} cfg={cfg}")

    if ap50 - best_ap < 0.0005:  # 連續N組差異小於閾值
        stable_count += 1
    else:
        best_ap = ap50
        stable_count = 0
    if stable_count >= early_stop_patience:
        print(f"🟢 AP50 已收斂，提早停止於第 {i+1}/{len(param_space)} 組。")
        break

[VAL] AP50=0.86133 cfg={'T_rtd': 1.3, 'B_rtd_p': -0.25, 'W_rtd_p': 0.3, 'WBF_p': 0.55, 'VL': 1.0}
[VAL] AP50=0.86133 cfg={'T_rtd': 1.3, 'B_rtd_p': -0.25, 'W_rtd_p': 0.3, 'WBF_p': 0.55, 'VL': 1.2}
[VAL] AP50=0.86124 cfg={'T_rtd': 1.3, 'B_rtd_p': -0.25, 'W_rtd_p': 0.3, 'WBF_p': 0.6, 'VL': 1.0}
[VAL] AP50=0.86124 cfg={'T_rtd': 1.3, 'B_rtd_p': -0.25, 'W_rtd_p': 0.3, 'WBF_p': 0.6, 'VL': 1.2}
[VAL] AP50=0.86162 cfg={'T_rtd': 1.3, 'B_rtd_p': -0.25, 'W_rtd_p': 0.5, 'WBF_p': 0.55, 'VL': 1.0}
[VAL] AP50=0.86162 cfg={'T_rtd': 1.3, 'B_rtd_p': -0.25, 'W_rtd_p': 0.5, 'WBF_p': 0.55, 'VL': 1.2}
[VAL] AP50=0.86135 cfg={'T_rtd': 1.3, 'B_rtd_p': -0.25, 'W_rtd_p': 0.5, 'WBF_p': 0.6, 'VL': 1.0}
[VAL] AP50=0.86135 cfg={'T_rtd': 1.3, 'B_rtd_p': -0.25, 'W_rtd_p': 0.5, 'WBF_p': 0.6, 'VL': 1.2}


KeyboardInterrupt: 

In [ ]:
import json
BEST_NOW = sorted(results, key=lambda x: -x[0])[:2]
print(BEST_NOW)
with open(os.path.join(OUT_DIR, "best_now_partial.json"), "w") as f:
    json.dump({"best": [{"ap50": ap, "cfg": cfg} for ap, cfg in BEST_NOW]}, f, indent=2)


[(0.8616242960530219, {'T_rtd': 1.3, 'B_rtd_p': -0.25, 'W_rtd_p': 0.5, 'WBF_p': 0.55, 'VL': 1.2}), (0.8616228850089256, {'T_rtd': 1.3, 'B_rtd_p': -0.25, 'W_rtd_p': 0.5, 'WBF_p': 0.55, 'VL': 1.0})]


In [ ]:
# ---- 取前 2 名，存 JSON ----
results.sort(key=lambda x: -x[0])
best2 = results[:2]
print("Top-2 (AP50 on VAL):")
for ap, cfg in best2:
    print(f"  AP50={ap:.5f}  cfg={cfg}")

BEST_JSON = os.path.join(OUT_DIR, "best2_configs.json")
with open(BEST_JSON, "w") as wf:
    json.dump({"best": [{"ap50": ap, "cfg": cfg} for ap, cfg in best2]}, wf, indent=2)
print("Saved:", BEST_JSON)


Top-2 (AP50 on VAL):
  AP50=0.86162  cfg={'T_rtd': 1.3, 'B_rtd_p': -0.25, 'W_rtd_p': 0.5, 'WBF_p': 0.55, 'VL': 1.2}
  AP50=0.86162  cfg={'T_rtd': 1.3, 'B_rtd_p': -0.25, 'W_rtd_p': 0.5, 'WBF_p': 0.55, 'VL': 1.0}
Saved: /content/drive/MyDrive/ComputerVision_DL/hw2_longtail_od/submissions/best2_configs.json


In [ ]:
with open(BEST_JSON, "r") as rf:
    best2 = json.load(rf)["best"]

test_imgs = sorted(glob.glob(os.path.join(TEST_DIR, "*.jpg")) + glob.glob(os.path.join(TEST_DIR, "*.png")))
assert test_imgs, f"在 {TEST_DIR} 找不到測試影像"

for rank, item in enumerate(best2, start=1):
    cfg = item["cfg"]
    params = BASE_PARAMS.copy()
    params["T_BY_MODEL"] = {"yolo":1.0, "rtdetr":cfg["T_rtd"]}
    params["B_BY_MODEL_CLASS"] = {"yolo":{0:0,1:0,2:0,3:0}, "rtdetr":{0:-0.05,1:-0.05,2:cfg["B_rtd_p"],3:-0.10}}
    params["W_BY_MODEL_CLASS"] = {"yolo":{0:1,1:1,2:1,3:1}, "rtdetr":{0:0.8,1:0.7,2:cfg["W_rtd_p"],3:0.6}}
    params["WBF_IOU_PERSON"] = cfg["WBF_p"]
    params["VAR_LAMBDA"]     = cfg["VL"]

    ts = time.strftime("%Y%m%d-%H%M%S")
    csv_path = os.path.join(OUT_DIR, f"submission_best{rank}_{ts}.csv")
    with open(csv_path, "w", newline="") as f:
        writer = csv.writer(f); writer.writerow(["Image_ID","PredictionString"])
        for i, pth in enumerate(test_imgs, start=1):
            preds = predict_one_image(pth, params)  # [(c,x1,y1,x2,y2,score)]
            parts = []
            im = cv2.imread(pth); H, W = im.shape[:2]
            for c, x1,y1,x2,y2, s in preds:
                x1 = clipf(float(x1), 0.0, float(W))
                y1 = clipf(float(y1), 0.0, float(H))
                w  = clipf(float(x2) - x1, 0.0, float(W) - x1)
                h  = clipf(float(y2) - y1, 0.0, float(H) - y1)
                parts += [f"{s:.6f}", f"{x1:.1f}", f"{y1:.1f}", f"{w:.1f}", f"{h:.1f}", str(int(c))]
            writer.writerow([i, " ".join(parts)])
    print(f"✅ 輸出提交檔：{csv_path}")

✅ 輸出提交檔：/content/drive/MyDrive/ComputerVision_DL/hw2_longtail_od/submissions/submission_best1_20251028-062229.csv
